In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## 1. Transformer Block

In [2]:
class TransformerBlock(nn.Module):
    """
    Transformer 블록은 Self-Attention과 Feed Forward Network를 결합한 구성요소입니다.
    주요 구성 요소:
      - Multi-Head Attention: 입력 시퀀스 내 토큰 간 관계를 여러 관점에서 학습합니다.
      - Feed Forward Network: 두 개의 선형 계층과 ReLU 활성화를 통해 비선형 변환을 수행합니다.
      - Layer Normalization: 각 서브레이어 후에 적용되어 학습을 안정화합니다.
      - Residual Connection: 입력과 서브레이어 출력을 더하여 깊은 네트워크의 학습을 용이하게 합니다.
    """
    def __init__(self, embed_dim, num_heads, ff_dim):
        super(TransformerBlock, self).__init__()
        # PyTorch의 nn.MultiheadAttention는 버전 1.9부터 batch_first 인자를 지원합니다.
        self.attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
        self.layernorm1 = nn.LayerNorm(embed_dim, eps=1e-6)
        self.layernorm2 = nn.LayerNorm(embed_dim, eps=1e-6)
        
    def forward(self, x):
        # Self-Attention: query, key, value 모두 동일한 x를 사용합니다.
        attn_output, _ = self.attn(x, x, x)
        # 첫 번째 residual connection + Layer Normalization
        x = self.layernorm1(x + attn_output)
        # Feed Forward Network 처리
        ffn_output = self.ffn(x)
        # 두 번째 residual connection + Layer Normalization
        x = self.layernorm2(x + ffn_output)
        return x

## 2. 토큰과 위치 임베딩 레이어

In [4]:
class TokenAndPositionEmbedding(nn.Module):
    """
    토큰과 위치 정보를 임베딩하는 레이어입니다.
    
    Transformer에서는 각 토큰을 임베딩한 값과 해당 위치의 임베딩을 더하여 최종 임베딩을 생성합니다.
    """
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)
        self.pos_emb = nn.Embedding(num_embeddings=maxlen, embedding_dim=embed_dim)
        self.maxlen = maxlen
        
    def forward(self, x):
        # x: (batch_size, seq_len)
        batch_size, seq_len = x.size()
        # 0부터 seq_len-1까지의 위치 인덱스 생성 후 임베딩
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)  # (1, seq_len)
        positions = self.pos_emb(positions)  # (1, seq_len, embed_dim)
        # 토큰 임베딩
        x = self.token_emb(x)  # (batch_size, seq_len, embed_dim)
        return x + positions  # 두 임베딩을 element-wise 합산

## 3. 인코더 구현


In [5]:
class Encoder(nn.Module):
    """
    Transformer 인코더는 입력 시퀀스를 임베딩하고 Transformer 블록을 통해 문맥화된 표현을 추출합니다.
    """
    def __init__(self, vocab_size, maxlen, embed_dim, num_heads, ff_dim):
        super(Encoder, self).__init__()
        self.embedding = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        
    def forward(self, x):
        # x: (batch_size, maxlen)
        x = self.embedding(x)            # (batch_size, maxlen, embed_dim)
        x = self.transformer_block(x)    # (batch_size, maxlen, embed_dim)
        return x

## 4. 디코더 구현

In [6]:
class Decoder(nn.Module):
    """
    Transformer 디코더는 타겟 시퀀스를 임베딩하고, Self-Attention을 적용한 후 인코더의 출력과 결합하여
    다음 토큰을 예측하는 로짓 값을 생성합니다.
    
    참고: 원본 TensorFlow 코드는 인코더-디코더 Attention으로 단순 Concatenation을 사용합니다.
    """
    def __init__(self, vocab_size, maxlen, embed_dim, num_heads, ff_dim):
        super(Decoder, self).__init__()
        self.embedding = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        # Concatenation 후 차원은 2 * embed_dim이 되므로, fc 레이어의 입력 차원도 2 * embed_dim입니다.
        self.fc = nn.Linear(2 * embed_dim, vocab_size)
        
    def forward(self, target, encoder_outputs):
        # target: (batch_size, maxlen), encoder_outputs: (batch_size, maxlen, embed_dim)
        x = self.embedding(target)            # (batch_size, maxlen, embed_dim)
        x = self.transformer_block(x)         # (batch_size, maxlen, embed_dim)
        # 인코더 출력과 Concatenation (마지막 차원 기준)
        x = torch.cat([x, encoder_outputs], dim=-1)  # (batch_size, maxlen, 2 * embed_dim)
        # Dense 층을 통해 vocab_size 차원의 로짓 생성
        x = self.fc(x)  # (batch_size, maxlen, vocab_size)
        return x

## 5. 전체 Transformer 모델 구성

In [8]:
class TransformerModel(nn.Module):
    """
    Transformer 모델의 전체 아키텍처를 구성합니다.
      - 인코더와 디코더를 연결하여 소스 시퀀스와 타겟 시퀀스를 처리합니다.
    """
    def __init__(self, vocab_size, maxlen, embed_dim, num_heads, ff_dim):
        super(TransformerModel, self).__init__()
        self.encoder = Encoder(vocab_size, maxlen, embed_dim, num_heads, ff_dim)
        self.decoder = Decoder(vocab_size, maxlen, embed_dim, num_heads, ff_dim)
        
    def forward(self, encoder_input, decoder_input):
        # encoder_input: (batch_size, maxlen), decoder_input: (batch_size, maxlen)
        encoder_output = self.encoder(encoder_input)              # (batch_size, maxlen, embed_dim)
        decoder_output = self.decoder(decoder_input, encoder_output)  # (batch_size, maxlen, vocab_size)
        return decoder_output

## 6. 모델 생성 및 학습 루프

In [19]:
# 하이퍼파라미터 설정
vocab_size = 10000   # 어휘 사전 크기
maxlen = 100         # 최대 시퀀스 길이
embed_dim = 256      # 임베딩 차원
num_heads = 8        # 멀티헤드 어텐션 헤드 수
ff_dim = 512         # 피드포워드 네트워크 은닉층 크기
batch_size = 32
epochs = 10

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = TransformerModel(vocab_size, maxlen, embed_dim, num_heads, ff_dim).to(device)

cuda


In [21]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [22]:
# 테스트용 무작위 데이터 생성
num_samples = 1000
input_data = torch.randint(0, vocab_size, (num_samples, maxlen), device=device)
target_data = torch.randint(0, vocab_size, (num_samples, maxlen), device=device)

model.train()
for epoch in range(epochs):
    permutation = torch.randperm(num_samples)
    epoch_loss = 0.0
    for i in range(0, num_samples, batch_size):
        indices = permutation[i:i+batch_size]
        encoder_input = input_data[indices]
        decoder_input = input_data[indices]  # 예제에서는 encoder와 decoder 입력 동일
        targets = target_data[indices]
        
        optimizer.zero_grad()
        outputs = model(encoder_input, decoder_input)  # (batch_size, maxlen, vocab_size)
        # nn.CrossEntropyLoss는 (N, C) 모양의 입력을 요구하므로 flatten 처리
        outputs = outputs.view(-1, vocab_size)  # (batch_size*maxlen, vocab_size)
        targets = targets.view(-1)              # (batch_size*maxlen)
        
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/10, Loss: 300.0681
Epoch 2/10, Loss: 268.0348
Epoch 3/10, Loss: 240.8848
Epoch 4/10, Loss: 212.1092
Epoch 5/10, Loss: 179.5875
Epoch 6/10, Loss: 143.8515
Epoch 7/10, Loss: 107.2166
Epoch 8/10, Loss: 70.9745
Epoch 9/10, Loss: 41.3296
Epoch 10/10, Loss: 21.5266


## 7. 인퍼런스 및 성능 평가

In [23]:
# 학습된 모델을 사용하여 몇 개의 샘플에 대해 인퍼런스를 수행합니다.  
# 각 샘플에 대해 입력 시퀀스와 모델이 예측한 토큰 ID를 출력합니다.

model.eval()
with torch.no_grad():
    # 인퍼런스용 테스트 샘플 5개 생성
    test_samples = 5
    test_input = torch.randint(0, vocab_size, (test_samples, maxlen), device=device)
    # 예제에서는 encoder와 decoder 입력으로 동일한 데이터를 사용
    test_output = model(test_input, test_input)  # (test_samples, maxlen, vocab_size)
    # 각 위치별로 가장 높은 로짓값을 갖는 토큰 ID 선택
    predicted_tokens = torch.argmax(test_output, dim=-1)
    
    # 결과 출력
    for i in range(test_samples):
        print(f"Sample {i+1}:")
        print("입력 토큰:  ", test_input[i].cpu().numpy())
        print("예측 토큰:", predicted_tokens[i].cpu().numpy())
        print()

Sample 1:
입력 토큰:   [4430 6560 3984 3270 2717 5945  631 3511   57 3887 9047 7297 8474 4169
 8755 2324 7598 2718  807 5807 8295 5189 7355 9182 9660 1860 2840 6814
 6494 4304 2801 5279 9849 3203 2578 5904 3426 7495 3181 5220 4929 9596
 8660  274 6170 8893 5661 9227 7698 1272 9858 1369   17 6946 2487 2085
 1678 6026 5101 6419 2327 1448   71 9899 4870 7871  524 6995 5686 7577
 2085 1803 3513 2418 6322 3581 7320 5901 7994  613 4418 4600  501 2490
 4943 9596 5762 6948 7260  882 4469 8471 1655 4266 3302 6345 6428 6778
 1813 9600]
예측 토큰: [6019 6918 7450 4880 9111  188 6112 4936 6215 7589 3080 9129 7686 9066
 9047  181 4798 3024 6030 8030 2388 4457 5096 2224 5316 4958 2001 7300
 5698 7447 1600 3729 6370 3125 3339 4349 8840 6693 5391 2682 2471 2101
 3250  506  831 2767 7119 5627 3844  117 1671 5258 6119 1342  656 2232
 1824 1717 1254 3895 8127 6253 6815 9432 1366 9763 9606 6413 9397 2805
 4545 9867 6740 3896 1400 7198 8485 5941 4187 3663 3794 1002 7068 5798
 7224 3577 9641 5777 5054 5838 2317   6